In [ ]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [34]:
import tensorflow as tf
from sklearn import cross_validation
from sklearn import utils as ut
import csv
import numpy as np
import os.path as path
from PIL import Image
from skimage.color import rgb2gray

In [35]:
# Load data
dataDir = './data'
featuresDir = './data/SampleFoodClassifier_Norm_100'

with open('./sample_food_no_food.csv') as f:
    food_no_food = [{k: v for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    

data_ids = [element['id'] for element in food_no_food]

labels = [int(element['is_food']) for element in food_no_food]
data = [rgb2gray(np.array(Image.open(path.join(featuresDir, element)))) for element in data_ids]

In [37]:
# Split training data in a train set and a test set. The test set will containt 20% of the total
x_train, x_test, y_train, y_test = cross_validation.train_test_split(data, labels, test_size=0.25, random_state=6)

# Parameters
learning_rate = 0.001
training_size = 10
training_iters = 30

# Network Parameters
n_input = 10000
n_classes = len(set(y_train))
dropout = 0.75 # Dropout, probability to keep units

# print 'Input vector size', n_input, 'train shape', np.array(x_train).shape , 'number of classes', n_classes

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [38]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    # !!!!!! IMPORTANT RESHAPE !!!!!!!!!!!!!! IMPORTANT RESHAPE !!!!!!!!!!!!!! IMPORTANT RESHAPE !!!!!!!!
    x = tf.reshape(x, shape=[-1, 100, 100, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    print conv2.get_shape()

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [39]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([25*25*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

(?, 25, 25, 64)


In [40]:
y_train_temp = []

for element in y_train:
    temp = [0]*len(set(y_train))
    temp[element] = 1
    y_train_temp.append(temp)
    
y_train = np.reshape(y_train_temp,(len(y_train_temp), -1))

y_test_temp = []

for element in y_test:
    temp = [0]*len(set(y_test))
    temp[element] = 1
    y_test_temp.append(temp)
    
y_test = np.reshape(y_test_temp,(len(y_test_temp), -1))

In [41]:
x_train = np.reshape(x_train, (len(x_train), -1))

In [42]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Keep training until reach max iterations
    for epoch in range(training_iters):
        # Fit training using batch data
        
        ix = utils.shuffle(x_train, n_samples=training_size, random_state=epoch)
        iy = utils.shuffle(y_train, n_samples=training_size, random_state=epoch)
        
        sess.run(optimizer, feed_dict={x: ix, y: iy, keep_prob: 1.})
        # Compute average loss
        loss, acc = sess.run([cost, accuracy], feed_dict={x: ix, y: iy, keep_prob: 1.})
        # Display logs per epoch step
        print "Iter " + str(epoch) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
    print "Optimization Finished!"

    # Calculate accuracy for 256 mnist test images
    
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: np.reshape(x_test, (len(x_test), -1)), y: y_test, keep_prob: 1.})

Iter 0, Minibatch Loss= 27875.005859, Training Accuracy= 0.90000
Iter 1, Minibatch Loss= 97098.000000, Training Accuracy= 0.80000
Iter 2, Minibatch Loss= 38417.812500, Training Accuracy= 0.90000
Iter 3, Minibatch Loss= 42704.367188, Training Accuracy= 0.60000
Iter 4, Minibatch Loss= 67146.421875, Training Accuracy= 0.70000
Iter 5, Minibatch Loss= 129349.546875, Training Accuracy= 0.30000
Iter 6, Minibatch Loss= 103952.265625, Training Accuracy= 0.40000
Iter 7, Minibatch Loss= 9193.427734, Training Accuracy= 0.70000
Iter 8, Minibatch Loss= 68252.500000, Training Accuracy= 0.70000
Iter 9, Minibatch Loss= 54558.539062, Training Accuracy= 0.80000
Iter 10, Minibatch Loss= 142457.781250, Training Accuracy= 0.70000
Iter 11, Minibatch Loss= 62814.625000, Training Accuracy= 0.80000
Iter 12, Minibatch Loss= 67937.078125, Training Accuracy= 0.80000
Iter 13, Minibatch Loss= 80565.265625, Training Accuracy= 0.70000
Iter 14, Minibatch Loss= 17891.644531, Training Accuracy= 0.90000
Iter 15, Minibatch

10